In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin
from sklearn.metrics import mean_squared_error, r2_score
import math
from scipy.stats import norm

In [3]:
df = pd.read_csv('/Users/sunyuanqi/Desktop/AmesHousingblanksremoved.csv')
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Enclosed Porch,Screen Porch,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,120,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,0,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,0,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [4]:
df.describe()

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,Screen Porch,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2882.000000,2.882000e+03,2882.000000,2440.000000,2882.000000,2882.000000,2882.000000,2882.000000,2882.000000,2860.000000,...,2881.000000,2881.000000,2882.000000,2882.000000,2882.000000,2882.000000,2882.000000,2882.000000,2882.000000,2882.000000
mean,1487.273074,7.147722e+08,57.404580,69.224590,10128.238723,6.095767,5.562457,1971.304650,1984.242540,101.511538,...,1.766053,472.747657,93.612422,47.730396,23.047536,16.066273,50.212006,6.247051,2007.753643,180891.681818
std,835.762726,1.887594e+08,42.684096,23.365335,7872.324219,1.418082,1.108995,30.348265,20.885279,178.609436,...,0.764290,216.025360,125.901955,67.760009,63.982497,56.297538,569.613568,2.721104,1.296015,80354.030005
min,1.000000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,769.250000,5.284582e+08,20.000000,58.000000,7436.500000,5.000000,5.000000,1953.000000,1965.000000,0.000000,...,1.000000,320.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129050.000000
50%,1489.500000,5.354541e+08,50.000000,68.000000,9422.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,...,2.000000,480.000000,0.000000,27.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,160000.000000
75%,2209.750000,9.071851e+08,70.000000,80.000000,11524.500000,7.000000,6.000000,2001.000000,2004.000000,163.250000,...,2.000000,576.000000,168.000000,70.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,213687.500000
max,2930.000000,1.007100e+09,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,5.000000,1488.000000,1424.000000,742.000000,1012.000000,576.000000,17000.000000,12.000000,2010.000000,755000.000000


In [5]:
df_converted = pd.get_dummies(df)
df_converted.head()

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_Abnorml,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,1,526301100,20,141.0,31770,6,5,1960,1960,112.0,...,0,0,0,1,0,0,0,0,1,0
1,2,526350040,20,80.0,11622,5,6,1961,1961,0.0,...,0,0,0,1,0,0,0,0,1,0
2,3,526351010,20,81.0,14267,6,6,1958,1958,108.0,...,0,0,0,1,0,0,0,0,1,0
3,4,526353030,20,93.0,11160,7,5,1968,1968,0.0,...,0,0,0,1,0,0,0,0,1,0
4,5,527105010,60,74.0,13830,5,5,1997,1998,0.0,...,0,0,0,1,0,0,0,0,1,0


In [6]:
df_filled=df_converted.fillna(0)

X=df_filled.drop(['SalePrice'],1)
y = df_filled['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


def getDummy(df):
    df=pd.get_dummies(df)
    return df

# One Hot Encoding
X_train=getDummy(X_train)
X_test=getDummy(X_test)

# Perform Lasso to remove less performing variables
lasso = Lasso()

lasso.fit(X_train,y_train)

X_train=X_train.iloc[:,lasso.coef_!=0]
X_test=X_test.iloc[:,lasso.coef_!=0]
lasso = Lasso().fit(X_train,y_train)
print("Training set score: {:.2f}".format(lasso.score(X_train,y_train)))
print("Test set score: {:.2f}".format(lasso.score(X_test,y_test)))
print("Number of features used: {}".format(np.sum(lasso.coef_!=0)))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Training set score: 0.94
Test set score: 0.92
Number of features used: 274


In [7]:
y_pred = lasso.predict(X_test)
# The mean square error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
#R^2 score
print('R^2 score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 455989379.88
R^2 score: 0.92


In [8]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_error

print('MAE:\t$%.2f' % mean_absolute_error(y_test, y_pred))
print('MSLE:\t%.5f' % mean_squared_log_error(y_test, y_pred))

MAE:	$14919.17
MSLE:	0.02131
